# 04_test_retrieval
Phase 1 testing for chunking, embedding, retrieval, and generation

In [1]:
import sys
import os

sys.path.append(os.path.abspath('../src'))

In [2]:
from core.retriever import BM25Retriever
bm25 = BM25Retriever.load("models/bm25_index.pkl")

ModuleNotFoundError: No module named 'src'

In [2]:

import json
from core.chunking import MarkdownChunker

chunker = MarkdownChunker()
chunks = []
with open("../data/processed/youmed_articles.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        doc = json.loads(line)
        chunks.extend(chunker.chunk_document(doc["content"], doc["metadata"]))

print(f"Chunks: {len(chunks)}")

Chunks: 55973


In [3]:
from core.embedding import ONNXEmbedding

embedding_model = ONNXEmbedding("models/bge-m3-onnx")
# embeddings = embedding_model.embed_batch([c.enriched_content for c in chunks])
# print(embeddings.shape)

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="YOUR_API_KEY")

indexes = pc.list_indexes()
print(indexes)


[{
    "name": "youmed-articles",
    "metric": "cosine",
    "host": "youmed-articles-abpdtub.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}]


In [ ]:
from db.vector_store import PineconeStore, QdrantStore

# store = QdrantStore(url="http://localhost:6333")
# store.create_collection(name="youmed_articles", dimension=embeddings.shape[1])
# store.upsert(chunks, embeddings)

# store = PineconeStore(api_key="YOUR_API_KEY", index_name="youmed_articles")
store = PineconeStore(api_key="YOUR_API_KEY", index_name="youmed-articles")

In [6]:
# def load_all_chunks_from_pinecone(index, namespace=None):
#     from core.chunking import Chunk

#     chunks = []
#     next_token = None

#     while True:
#         res = list(index.list(limit=100, pagination_token=next_token, namespace=namespace))
#         ids = [v["id"] for v in res]
#         if not ids:
#             break

#         fetched = index.fetch(ids=ids, namespace=namespace)

#         for vid, v in fetched["vectors"].items():
#             md = v["metadata"]
#             chunks.append(
#                 Chunk(
#                     chunk_id=vid,
#                     enriched_content=md["enriched_content"],
#                     metadata=md,
#                     section=md.get("section"),
#                     subsection=md.get("subsection"),
#                 )
#             )

#         next_token = res.get("pagination_token")
#         if not next_token:
#             break

#     return chunks
# chunks1 = load_all_chunks_from_pinecone(store.index)

In [6]:
from core.chunking import Chunk

def load_chunks_from_pinecone(index):
    chunks = []

    res = index.query(
        vector=[0.0]*1024,   # dummy
        top_k=10000,
        include_metadata=True
    )

    for m in res["matches"]:
        chunks.append(
            Chunk(
                chunk_id=m["id"],
                enriched_content=m["metadata"]["enriched_content"],
                metadata=m["metadata"],
                section=m["metadata"].get("section"),
                subsection=m["metadata"].get("subsection"),
            )
        )

    return chunks
chunks1 = load_chunks_from_pinecone(store.index)

In [7]:
from core.retriever import BM25Retriever, HybridRetriever
from core.reranker import ONNXReranker


bm25 = BM25Retriever(chunks1)
reranker = ONNXReranker("models/bge-reranker-v2-m3-onnx")
retriever = HybridRetriever(store, bm25, embedding_model, reranker)

query = "Bệnh Addison có triệu chứng gì?"
results = retriever.retrieve(query, top_k=5)
for i, chunk in enumerate(results, 1):
    print(chunk.enriched_content)

Addison | Triệu chứng của bệnh Addison là gì? | Theo thời gian, bệnh Addison hay còn được gọi là suy thượng thận nguyên phát, sẽ dẫn đến các triệu chứng sau:
* Mệt mỏi mãn tính và cảm giác yếu cơ.
* Mất cảm giác ngon miệng, không có khả năng tiêu hóa thức ăn và giảm cân.
* Huyết áp thấp(hạ huyết áp) giảm nhiều hơn khi đứng. Điều này gây ra chóng mặt, đôi khi đến mức ngất xỉu.
* Xuất hiện các vết nám, sạm đen và tàn nhang trên da.
* Da sẫm màu đặc biệt xảy ra nhiều ở vùng trán, đầu gối và khuỷu tay hoặc dọc theo sẹo, nếp gấp da và nếp nhăn (như ở lòng bàn tay).
* Bất thường lượng đường trong máu, như lượng đường trong máu thấp nguy hiểm (hạ
Addison | Triệu chứng của bệnh Addison là gì? | máu, như lượng đường trong máu thấp nguy hiểm (hạ đường huyết).
* Buồn nôn, nônvà tiêu chảy.
* Không có khả năng đối phó với căng thẳng.
* Tâm trạng buồn bực, khó chịu và trầm cảm.
* Không thích nghi được với cảm giác nóng hoặc lạnh
* Thèm đồ ăn mặn.

Vì các triệu chứng của bệnh Addison tiến triển chậm,

In [ ]:
from core.generator import MedicalRAGGenerator

generator = MedicalRAGGenerator(openai_api_key="YOUR_API_KEY")
answer = generator.generate(query, results)
print(answer["answer"])

Bệnh Addison, hay suy thượng thận nguyên phát, có các triệu chứng chính sau:
- Mệt mỏi mãn tính và cảm giác yếu cơ.
- Mất cảm giác ngon miệng, không có khả năng tiêu hóa thức ăn và giảm cân.
- Huyết áp thấp, gây chóng mặt khi đứng.
- Xuất hiện các vết nám, sạm đen và tàn nhang trên da.
- Da sẫm màu, đặc biệt ở vùng trán, đầu gối và khuỷu tay.
- Bất thường lượng đường trong máu, như hạ đường huyết.
- Buồn nôn, nôn và tiêu chảy.
- Không thích nghi được với cảm giác nóng hoặc lạnh.
- Thèm đồ ăn mặn.

Nếu bạn nghi ngờ mình có các triệu chứng này, hãy tham khảo ý kiến của bác sĩ để được tư vấn và chẩn đoán chính xác.


In [9]:
print(answer)

{'answer': 'Bệnh Addison, hay suy thượng thận nguyên phát, có các triệu chứng chính sau:\n- Mệt mỏi mãn tính và cảm giác yếu cơ.\n- Mất cảm giác ngon miệng, không có khả năng tiêu hóa thức ăn và giảm cân.\n- Huyết áp thấp, gây chóng mặt khi đứng.\n- Xuất hiện các vết nám, sạm đen và tàn nhang trên da.\n- Da sẫm màu, đặc biệt ở vùng trán, đầu gối và khuỷu tay.\n- Bất thường lượng đường trong máu, như hạ đường huyết.\n- Buồn nôn, nôn và tiêu chảy.\n- Không thích nghi được với cảm giác nóng hoặc lạnh.\n- Thèm đồ ăn mặn.\n\nNếu bạn nghi ngờ mình có các triệu chứng này, hãy tham khảo ý kiến của bác sĩ để được tư vấn và chẩn đoán chính xác.', 'sources': [{}, {}, {}, {}, {}], 'tokens_used': 1968, 'cost': 0.002952}
